In [ ]:
import boto3
import json

# login for boto3 credentials
boto3.setup_default_session(profile_name='root-AWSAdministratorAccess')
client = boto3.client('bedrock')
bedrock_runtime_client = boto3.client('bedrock-runtime')

In [ ]:
# models details
response = client.list_foundation_models()
models = response['modelSummaries']
models_details = [f"{model['providerName']}, {model['modelName']}, {model['modelId']}, {model['modelArn']}, inputModalities: {','.join(model['inputModalities'])}, outputModalities: {','.join(model['outputModalities'])}" for model in models]
models_details.sort()
print('\n'.join(models_details))

In [ ]:
prompt  = 'list three names of people'
body = json.dumps(
                {
                    "prompt": prompt,
                    "maxTokens": 500,
                    "temperature": 0,
                    "stopSequences": [],
                })
response = bedrock_runtime_client.invoke_model(modelId='ai21.j2-ultra', body=body)
response_body = json.loads(response.get("body").read())
response_body

In [ ]:
# stream response
# Define prompt and model parameters
prompt_data = """Write an essay about why someone should drink coffee"""

text_gen_config = {
    "maxTokenCount": 1000,
    "stopSequences": [], 
    "temperature": 0,
    "topP": 0.9
}

body = json.dumps({
    "inputText": prompt_data,
    "textGenerationConfig": text_gen_config  
})

model_id = 'amazon.titan-tg1-large'
accept = 'application/json' 
content_type = 'application/json'

#invoke the model with a streamed response 
response = bedrock_runtime_client.invoke_model_with_response_stream(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)

for event in response['body']:
    data = json.loads(event['chunk']['bytes'])
    print(data['outputText'])

In [ ]:
# titan embedding - embed `prompt` and print

# Define prompt and model parameters
prompt_data = """Write me a poem about apples"""

body = json.dumps({
    "inputText": prompt_data,
})

model_id = 'amazon.titan-embed-g1-text-02' #look for embeddings in the modelID
accept = 'application/json' 
content_type = 'application/json'

# Invoke model 
response = bedrock_runtime_client.invoke_model(
    body=body, 
    modelId=model_id, 
    accept=accept, 
    contentType=content_type
)

# Print response
response_body = json.loads(response['body'].read())
embedding = response_body.get('embedding')

#Print the Embedding

print(embedding)